In [1]:
from Assessment.Credibility_assessment import run_credibility_assessment
from Assessment.Performance_assessment import run_performance_assessment
from Assessment.run_assessment import prepare_data_and_model, test_get_stocks_recommendation
from Assessment.utils import parse_args, normalize_assessment_results_list

D:\ProgramFiles\Miniconda\envs\qlib\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\ProgramFiles\Miniconda\envs\qlib\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
D:\ProgramFiles\Miniconda\envs\qlib\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import argparse
from Assessment.utils import ParseConfigFile
import sys
def parse_args():
    parser = argparse.ArgumentParser()

    # model
    parser.add_argument('--model_name', default='relation_GATs')
    parser.add_argument('--model_path', default='D:\Research\Fintech\K-Quant\parameter')
    parser.add_argument('--num_relation', type= int, default=134)
    parser.add_argument('--d_feat', type=int, default=6)
    parser.add_argument('--hidden_size', type=int, default=128)
    parser.add_argument('--num_layers', type=int, default=2)
    parser.add_argument('--dropout', type=float, default=0.1)
    parser.add_argument('--K', type=int, default=1)
    parser.add_argument('--loss_type', default='')
    parser.add_argument('--config', action=ParseConfigFile, default='')
    # for ts lib model
    parser.add_argument('--seq_len', type=int, default=60)
    parser.add_argument('--moving_avg', type=int, default=21)
    parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
    parser.add_argument('--embed', type=str, default='timeF',
                        help='time features encoding, options:[timeF, fixed, learned]')
    parser.add_argument('--freq', type=str, default='b',
                        help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
    parser.add_argument('--distil', action='store_false',
                        help='whether to use distilling in encoder, using this argument means not using distilling',
                        default=False)
    parser.add_argument('--factor', type=int, default=1, help='attn factor')
    parser.add_argument('--n_heads', type=int, default=1, help='num of heads')
    parser.add_argument('--d_ff', type=int, default=64, help='dimension of fcn')
    parser.add_argument('--activation', type=str, default='gelu', help='activation')
    parser.add_argument('--e_layers', type=int, default=8, help='num of encoder layers')
    parser.add_argument('--top_k', type=int, default=5, help='for TimesBlock')
    parser.add_argument('--pred_len', type=int, default=-1, help='the length of pred squence, in regression set to -1')
    parser.add_argument('--de_norm', default=True, help='de normalize or not')

    # data
    parser.add_argument('--data_set', type=str, default='csi360')
    parser.add_argument('--target', type=str, default='t+0')
    parser.add_argument('--pin_memory', action='store_false', default=True)
    parser.add_argument('--batch_size', type=int, default=-1)  # -1 indicate daily batch
    parser.add_argument('--least_samples_num', type=float, default=1137.0)
    parser.add_argument('--label', default='')  # specify other labels
    parser.add_argument('--start_date', default='2019-01-01')
    parser.add_argument('--end_date', default='2019-01-05')

    # input for csi 300
    parser.add_argument('--data_root', default='D:\Research\Fintech\K-Quant\Data')
    parser.add_argument('--market_value_path', default= 'D:\Research\Fintech\K-Quant\Data\csi300_market_value_07to20.pkl')

    parser.add_argument('--stock2concept_matrix', default='D:\ProjectCodes\K-Quant\Data\csi300_stock2concept.npy')


    parser.add_argument('--stock2stock_matrix', default='D:\Research\Fintech\K-Quant\Data\csi300_multi_stock2stock_hidy_2024.npy')
    parser.add_argument('--stock_index', default='D:\Research\Fintech\K-Quant\Data\csi300_stock_index_2024.npy')
    parser.add_argument('--model_dir', default='D:\Research\Fintech\K-Quant\parameter')
    parser.add_argument('--overwrite', action='store_true', default=False)
    parser.add_argument('--device', default='cpu')
    filtered_args = [arg for arg in sys.argv if not arg.startswith('-f') and not arg.endswith('.json')]
    args = parser.parse_args(filtered_args[1:])  # Skip the script name
    return args




In [3]:
args = parse_args()

model_list = ['LSTM', 'GRU', 'MLP', 'NRSR', 'relation_GATs']
# model_list = ['NRSR']
explanation_model = "inputGradientExplainer"
seq_len_list = [30, 60]

args.start_date = '2022-03-01'
args.end_date = '2022-03-10'

args.seq_len = 60
c_a_r_list = []
for seq_len in seq_len_list:
    for model in model_list:
        h_p_dict = {
            "prediction_model": model,
            "explanation_model": explanation_model,
            "start_date": args.start_date,
            "end_date": args.end_date,
            "seq_len": seq_len
        }
        args.model_name = model
        args.seq_len = seq_len

        data_loader, param_dict, model = prepare_data_and_model(args)
        credibility_assessment_results_dict = run_credibility_assessment(param_dict, data_loader, model,
                                                                         explanation_model)

        c_a_r_list.append((h_p_dict, credibility_assessment_results_dict))

n_c_a_r_list = normalize_assessment_results_list(c_a_r_list, num_selection = 5)

print(n_c_a_r_list)

predict in  LSTM


100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 113.05it/s]


predict in  GRU


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 14.44it/s]


predict in  MLP


...

predict in  NRSR


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 10.49it/s]


predict in  relation_GATs


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 10.97it/s]

[({'prediction_model': 'LSTM', 'explanation_model': 'inputGradientExplainer', 'start_date': '2022-03-01', 'end_date': '2022-03-10', 'seq_len': 30}, {'可靠性得分': 0.9999999999999999, '稳定性得分': 0.20558592917100738, '鲁棒性得分': 0.5382398876648296, '透明性得分': 0.0, '解释效果得分': 0.0}), ({'prediction_model': 'GRU', 'explanation_model': 'inputGradientExplainer', 'start_date': '2022-03-01', 'end_date': '2022-03-10', 'seq_len': 30}, {'可靠性得分': 0.8333445472038007, '稳定性得分': 0.0, '鲁棒性得分': 0.1621049921190192, '透明性得分': 0.0, '解释效果得分': 0.0}), ({'prediction_model': 'MLP', 'explanation_model': 'inputGradientExplainer', 'start_date': '2022-03-01', 'end_date': '2022-03-10', 'seq_len': 30}, {'可靠性得分': 0.0, '稳定性得分': 1.0, '鲁棒性得分': 1.0, '透明性得分': 0.0, '解释效果得分': 0.0}), ({'prediction_model': 'NRSR', 'explanation_model': 'inputGradientExplainer', 'start_date': '2022-03-01', 'end_date': '2022-03-10', 'seq_len': 30}, {'可靠性得分': 0.800120663283993, '稳定性得分': 0.7980980405609168, '鲁棒性得分': 0.26870026714652084, '透明性得分': 1.0, '解释效果得分': 1.0

In [4]:
args.model_name = "NRSR"
explanation_model = "inputGradientExplainer"
args.seq_len = 60
args.num_recommendation_stocks = 3
data_loader, param_dict, model = prepare_data_and_model(args)
recommend_stocks_list = test_get_stocks_recommendation(param_dict, data_loader, model,
                                                       top_n=args.num_recommendation_stocks)  # 输出的是推荐的股票

print(recommend_stocks_list)

predict in  NRSR


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 10.45it/s]

['002791.SZ', '688169.SH', '002841.SZ']


In [5]:
select_dict_list = [
    {
        '600061.SH': 0.1,
        '601009.SH': 0.2,
        '601066.SH': 0.1,
        '600519.SH': 0.3,
        '600606.SH': 0.3
    },
    {
        '600061.SH': 0.2,
        '601009.SH': 0.2,
        '600887.SH': 0.4,
        '600132.SH': 0.2,
    },
    {
        '600010.SH': 0.8,
        '600132.SH': 0.1,
        '600489.SH': 0.1

    },
    {
        '600760.SH': 0.3,
        '600000.SH': 0.2,
        '600600.SH': 0.2,
        '601088.SH': 0.3

    },
    {
        '600837.SH': 0.7,
        '601009.SH': 0.2,
        '601066.SH': 0.1,

    },
    {
        '601009.SH': 0.1,
        '601066.SH': 0.5,
        '600132.SH': 0.4
    },
]

    # 下面开始计算性能评价
    #更新时间表述
args.start_date = args.start_date.replace('-', '')
args.end_date = args.end_date.replace('-', '')
args.return_preference = 2 # 输入回报偏好
args.risk_preference = 60 # 输入风险偏好
p_a_r_list = []
for select_dict in select_dict_list:
    h_p_dict = {
                    "select_dict": select_dict,
                    "return_preference": args.return_preference,
                    "seq_len": args.risk_preference,
                    "start_date": args.start_date,
                    "end_date": args.end_date,

                }
    performance_assessment_results_dict = run_performance_assessment(args, select_dict) # 输出性能得分
    print(performance_assessment_results_dict)
    p_a_r_list.append((h_p_dict, performance_assessment_results_dict))

n_p_a_r_list = normalize_assessment_results_list(p_a_r_list, num_selection=5)
print(n_p_a_r_list)

{'600061.SH': 0.1, '601009.SH': 0.2, '601066.SH': 0.1, '600519.SH': 0.3, '600606.SH': 0.3}
600061.SH 证券 48
601009.SH 银行 41
601066.SH 证券 48
600519.SH 酿酒行业 37
600606.SH 房地产开发 107


100%|████████████████████████████████████████████████████████████████████████████████| 107/107 [00:21<00:00,  4.94it/s]


------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
600061.SH: 近1天年化收益率位于同行业股票Top33%(16/48), 年化波动率位于同行业股票Top33% (16/48)。
600061.SH: 近5天年化收益率位于同行业股票Top66%(32/48), 年化波动率位于同行业股票Top54% (26/48)。
601009.SH: 近1天年化收益率位于同行业股票Top68%(28/41), 年化波动率位于同行业股票Top68% (28/41)。
601009.SH: 近5天年化收益率位于同行业股票Top21%(9/41), 年化波动率位于同行业股票Top51% (21/41)。
601066.SH: 近1天年化收益率位于同行业股票Top10%(5/48), 年化波动率位于同行业股票Top10% (5/48)。
601066.SH: 近5天年化收益率位于同行业股票Top93%(45/48), 年化波动率位于同行业股票Top75% (36/48)。
600519.SH: 近1天年化收益率位于同行业股票Top32%(12/37), 年化波动率位于同行业股票Top32% (12/37)。
600519.SH: 近5天年化收益率位于同行业股票Top13%(5/37), 年化波动率位于同行业股票Top45% (17/37)。
600606.SH: 近1天年化收益率位于同行业股票Top44%(48/107), 年化波动率位于同行业股票Top44% (48/107)。
600606.SH: 近5天年化收益率位于同行业股票Top84%(90/107), 年化波动率位于同行业股票Top9% (10/107)。
{'用户收益偏好得分': 0, '用户风险偏好得分': 8.401130712773167, '用户投资体验感得分': 5, '沪深300指数比较得分': 9.469759049082883, '中证500指数比较得分': 9.261513769552435

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:08<00:00,  4.53it/s]


------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
600061.SH: 近1天年化收益率位于同行业股票Top33%(16/48), 年化波动率位于同行业股票Top33% (16/48)。
600061.SH: 近5天年化收益率位于同行业股票Top66%(32/48), 年化波动率位于同行业股票Top54% (26/48)。
601009.SH: 近1天年化收益率位于同行业股票Top68%(28/41), 年化波动率位于同行业股票Top68% (28/41)。
601009.SH: 近5天年化收益率位于同行业股票Top21%(9/41), 年化波动率位于同行业股票Top51% (21/41)。
600887.SH: 近1天年化收益率位于同行业股票Top95%(100/105), 年化波动率位于同行业股票Top95% (100/105)。
600887.SH: 近5天年化收益率位于同行业股票Top50%(53/105), 年化波动率位于同行业股票Top27% (29/105)。
600132.SH: 近1天年化收益率位于同行业股票Top27%(10/37), 年化波动率位于同行业股票Top27% (10/37)。
600132.SH: 近5天年化收益率位于同行业股票Top97%(36/37), 年化波动率位于同行业股票Top91% (34/37)。
{'用户收益偏好得分': 0, '用户风险偏好得分': 8.235850929074163, '用户投资体验感得分': 5, '沪深300指数比较得分': 9.357245494132972, '中证500指数比较得分': 9.121042464519437, '基准比较得分': 6.357245494132973, '同行业股票比较对比得分': 49.94780018672677}
{'600010.SH': 0.8, '600132.SH': 0.1, '600489.SH': 0.1}
600010.SH 钢铁

...

------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
600760.SH: 近1天年化收益率位于同行业股票Top55%(19/34), 年化波动率位于同行业股票Top55% (19/34)。
600760.SH: 近5天年化收益率位于同行业股票Top8%(3/34), 年化波动率位于同行业股票Top67% (23/34)。
600000.SH: 近1天年化收益率位于同行业股票Top82%(34/41), 年化波动率位于同行业股票Top82% (34/41)。
600000.SH: 近5天年化收益率位于同行业股票Top70%(29/41), 年化波动率位于同行业股票Top41% (17/41)。
600600.SH: 近1天年化收益率位于同行业股票Top70%(26/37), 年化波动率位于同行业股票Top70% (26/37)。
600600.SH: 近5天年化收益率位于同行业股票Top59%(22/37), 年化波动率位于同行业股票Top94% (35/37)。
601088.SH: 近1天年化收益率位于同行业股票Top64%(22/34), 年化波动率位于同行业股票Top64% (22/34)。
601088.SH: 近5天年化收益率位于同行业股票Top38%(13/34), 年化波动率位于同行业股票Top61% (21/34)。
{'用户收益偏好得分': 0, '用户风险偏好得分': 7.64652934445396, '用户投资体验感得分': 5, '沪深300指数比较得分': 9.793487186501523, '中证500指数比较得分': 9.647301965946879, '基准比较得分': 6.793487186501522, '同行业股票比较对比得分': 30.283939022640002}
{'600837.SH': 0.7, '601009.SH': 0.2, '601066.SH': 0.1}
600837.SH 证券 48
601

100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [00:08<00:00,  4.98it/s]


------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
600837.SH: 近1天年化收益率位于同行业股票Top77%(37/48), 年化波动率位于同行业股票Top77% (37/48)。
600837.SH: 近5天年化收益率位于同行业股票Top56%(27/48), 年化波动率位于同行业股票Top39% (19/48)。
601009.SH: 近1天年化收益率位于同行业股票Top68%(28/41), 年化波动率位于同行业股票Top68% (28/41)。
601009.SH: 近5天年化收益率位于同行业股票Top21%(9/41), 年化波动率位于同行业股票Top51% (21/41)。
601066.SH: 近1天年化收益率位于同行业股票Top10%(5/48), 年化波动率位于同行业股票Top10% (5/48)。
601066.SH: 近5天年化收益率位于同行业股票Top93%(45/48), 年化波动率位于同行业股票Top75% (36/48)。
{'用户收益偏好得分': 0, '用户风险偏好得分': 8.643855937213651, '用户投资体验感得分': 5, '沪深300指数比较得分': 8.502664704709703, '中证500指数比较得分': 7.820508882514041, '基准比较得分': 5.502664704709703, '同行业股票比较对比得分': 40.29311399223146}
{'601009.SH': 0.1, '601066.SH': 0.5, '600132.SH': 0.4}
601009.SH 银行 41
601066.SH 证券 48
600132.SH 酿酒行业 37


100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:05<00:00,  6.37it/s]


------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
601009.SH: 近1天年化收益率位于同行业股票Top68%(28/41), 年化波动率位于同行业股票Top68% (28/41)。
601009.SH: 近5天年化收益率位于同行业股票Top21%(9/41), 年化波动率位于同行业股票Top51% (21/41)。
601066.SH: 近1天年化收益率位于同行业股票Top10%(5/48), 年化波动率位于同行业股票Top10% (5/48)。
601066.SH: 近5天年化收益率位于同行业股票Top93%(45/48), 年化波动率位于同行业股票Top75% (36/48)。
600132.SH: 近1天年化收益率位于同行业股票Top27%(10/37), 年化波动率位于同行业股票Top27% (10/37)。
600132.SH: 近5天年化收益率位于同行业股票Top97%(36/37), 年化波动率位于同行业股票Top91% (34/37)。
{'用户收益偏好得分': 0, '用户风险偏好得分': 7.605426677203649, '用户投资体验感得分': 5, '沪深300指数比较得分': 9.422893639743915, '中证500指数比较得分': 9.203477882688105, '基准比较得分': 6.422893639743916, '同行业股票比较对比得分': 40.29311399223146}
[({'select_dict': {'600061.SH': 0.1, '601009.SH': 0.2, '601066.SH': 0.1, '600519.SH': 0.3, '600606.SH': 0.3}, 'return_preference': 2, 'seq_len': 60, 'start_date': '20220301', 'end_date': '20220310'}, {'用户收益偏好得分': 